Attention Mechanism

고양이가 잔다 - > the cat sleeps

입력 : "고양이가 잔다"

[전처리] -> <start> 고양이가 잔다 <end>

[ENCODER] -> 각 단어마다 hidden state 생성

[ATTENTION] -> 어디에 집중할지 계산

[DECODER] -> 한 단어씩 생성

출력 : the cat sleeps

In [ ]:
# 원본
input = '고양이가 자요'
# 1단계 전처리
input = f'<start> {input} <end>'
# 2단계 토큰화(단어 -> 숫자)
word_dict = {
    '<pad>' : 0,
    '<start>' : 1,
    '<end>' : 2,
    '고양이가' : 3,
    '자요' : 4,
}

# 최대길이 5
정수_시퀀스 = [1,3,4,2,0]

In [ ]:
# STEP 1 : EMBEDDING(숫자 -> 벡터)
# 임베딩레이어 ============= 256차원
정수_시퀀스 = [1,3,4,2,0]
# Embedding....
임베딩_결과 = [
    [0.2, 0.5, ..., 0.3], #1 (<start>)의 벡터 (256)차원
    [0.2, 0.5, ..., 0.3], #3 (<start>)의 벡터 (256)차원
    [0.2, 0.5, ..., 0.3], #4 (<start>)의 벡터 (256)차원
    [0.2, 0.5, ..., 0.3], #2 (<start>)의 벡터 (256)차원
    [0.2, 0.5, ..., 0.3], #0 (<start>)의 벡터 (256)차원
]

In [ ]:
from tensorflow.keras.layers import LSTM
# STEP 2 : ENCODER(벡터 -> Hidden States)
# LSTM Encoder
# 각 타임스텝마다 hidden state 생성

# 초기상태
h0 = [0,0,...,0] # 512차원 영벡터
c0 = [0,0,...,0] # 512차원 영벡터

# ================= 타임스텝 1 : <start> 처리 ================
입력_t1 = [0.2,0.5, ..., 0.3] #  <start> 의 임베딩
h1, c1 = LSTM(입력_t1, h0, c0)
# h1 = [0.1, ......] 512차원

# ================= 타임스템 2 : 고양이가 처리 ================
입력_t2 = [0.2,0.5, ..., 0.3] # 고양이가 의 임베딩
h2, c2 = LSTM(입력_t2, h1, c1)

# ================= 타임스템 3 : 자요 처리 ================
입력_t3 = [0.2,0.5, ..., 0.3] # 자요 의 임베딩
h3, c3 = LSTM(입력_t3, h2, c2)

# ================= 타임스템 4 : <end> 처리 ================
입력_t4 = [0.2,0.5, ..., 0.3] # <end> 의 임베딩
h4, c4 = LSTM(입력_t4, h3, c3)

# ================= 타임스템 5 : 패딩 (무시) ================
입력_t5 = [0.0,0.0, ..., 0.0] 
h5, c5 = LSTM(입력_t5, h4, c4)
# h5는 사용 안함(마스킹)

# ================= 인코더의 출력 =================
encoder_output = [h1,h2,h3,h4,h5] # 모든 hideen state
# 형태 [1,5,512]   [배치, 시퀀스, units]
encoder_final_h = h4 # 마지막 hidden state(디코더 초기화용)
encoder_final_c = c4 # 마지막 cell state

# ================== 정리 ===================
# <start>  ---> h1
# 고양이가  ---> h2   "고양이" 정보를 압축
# 자요     ---> h3   "자요" 정보를 압축
# <end>    ---> h4

In [ ]:
# STEP 3 : Decoder의 시작 (첫 단어 생성)
# ============== 디코더 초기 상태 ===============
decoder_h = h4  # 인코더의 마지막 hidden state
decoder_c = c5

# ============= 첫번째 입력 : <start> 토큰 ============
decoder_input = [1]  # <start> 의 인덱스
decoder_input_embedding = [0.3, 0.1, ..., 0.2 ] #256차원


In [ ]:
# STEP 4 : ATTENTION 계산 (첫 번쨰 단어)
#### THE 를 생성하기 위해서 어디를 봐야 하나? ####
# query 준비
query = decoder_h # 512차원
# 의미 : 지금 뭘 번역하려고 하는가?

# key 준비
keys = [h1, h2, h3, h4] # 패딩을 제외한 인코더 출력
# 의미 : 각 입력 단어의 정보

# value 준비(keys와 동일)
values = [h1, h2, h3, h4] #패딩을 제외한 인코더 출력

# 1단계 score 계산
# W1 W2 V는 학습된 가중치 행렬
# h1 <start> 에 대한 score 계산
# h2 고양이가 에 대한 score 계산
# h3 자요 에 대한 score 계산
# h4 <end> 에 대한 score 계산
scores = [ 0.3,0.8,0.2,0.1]

# 2단계 Attention Weights 계산
# softmax 확률 분포 exp(scores)
exp_scores = [exp(0.3, exp(0.8)), ...] = [1.45, 2.23, 1.22, 1.11]
# sum
sum_exp = 1.45 + 2.23 + 1.22 + 1.11 = 5.91
# 정규화
attention_weight = [
    1.35 / 5.91 = 0.23, # <start>
    2.23 / 5.91 = 0.2, # 고양이가
    1.22 / 5.91 = 0.41, # 자요
    1.11 / 5.91 = 0.52, # <end> ---> 가중치가 가장 높음
]

# 3단계 context vector 계산
# 가중합으로 인코더 정보 통합
context_vector = [
    0.23 x h1 + , # <start>의 정보 23%
    0.2 x h2 + , # 고양이가 정보 20%
    0.41, # 자요 정보 41%
    0.52, # <end> 정보 52% ---> 가장 많이 참조
]

# 결과 512차원

# 입력단어  score   Attention weight    기여도
# ----      ----    -----              end에 대한 기여도가 가장 높게 나오고 ----> 집중


In [ ]:
# STEP 5 : DECODER 실행 첫 단어 생성
# context vector 와 입력을 결합해서 단어 생성

# 1단게 입력과 context 결합
decoder_input_emb = [0.2, ... , 0.2] # <start>의 임베딩(256차원)
context_vector = [0.32, ... , 0.22] 

# concatenate 이어붙이기
combined_input = [
    0.3,0.2,...,0.2, # 임베딩 256
    0.3,0.2,...,0.2, # context 512
]
# 형태: 256 + 512 768차원

# --- 2단계 LSTM 처리
decoder_output, new_h, new_c = LSTM(
    combined_input,
    decoder_h,
    decoder_c,
) 
#  ------- 3단계 : 단어 확률 분포 생성
# Dense layer : 512 + vocab_size(예 50)
logits = Dense(decoder_output)
# logis = [-0.5, 0.2, ...] 50개의 단어 점수

# softmax로 확률변환
probailities = softmax(logits)
# probailities = [0.0, 0.42, 0.52, ...]  인덱스 1 (the)가 가장 높게

# 4단계 ===== 가장 높은 확률의 단어 선택
predict_di = argmax(probailities) # 1

# 단어 사전에서 lookup
# 영어_단어_사전 = {

# }